In [1]:
import os
import pandas as pd

BASE_PATH = "C:/Users/aguil/OneDrive/Desktop/proyectos javier/Decision-Tree-Project-/models/Excel"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm.xlsx",
    "X_train_sin_outliers_norm.xlsx",
    "X_train_con_outliers_scal.xlsx",
    "X_train_sin_outliers_scal.xlsx"
]
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_excel(f"{BASE_PATH}/{path}")
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm.xlsx",
    "X_test_sin_outliers_norm.xlsx",
    "X_test_con_outliers_scal.xlsx",
    "X_test_sin_outliers_scal.xlsx"
]
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(f"{BASE_PATH}/{path}")
    )

y_train = pd.read_excel(f"{BASE_PATH}/y_train.xlsx")
y_test = pd.read_excel(f"{BASE_PATH}/y_test.xlsx")

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

results = []
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = DecisionTreeClassifier(random_state = 42)
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3
4
5


[{'train': 1.0, 'test': 0.7142857142857143},
 {'train': 1.0, 'test': 0.7272727272727273},
 {'train': 1.0, 'test': 0.7142857142857143},
 {'train': 1.0, 'test': 0.7272727272727273},
 {'train': 1.0, 'test': 0.7142857142857143},
 {'train': 1.0, 'test': 0.7272727272727273}]

In [10]:
best_dataset = 1 # Porque de results, el mayor accuracy en train y test es el de la posición 1

hyperparams = {
 
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier(random_state = 42)
grid = GridSearchCV(model, hyperparams, scoring = "accuracy")
y_train_fixed = y_train.values.ravel() if hasattr(y_train, "values") else y_train
grid.fit(TRAIN_DATASETS[best_dataset], y_train_fixed)
# Mostrar los mejores parámetros
print("Mejores parámetros:", grid.best_params_)
print("Mejor accuracy:", grid.best_score_)

Mejores parámetros: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10}
Mejor accuracy: 0.7459016393442622


C:\Users\aguil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
450 fits failed out of a total of 1350.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\aguil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\aguil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-

In [12]:
# 1️⃣ Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# 2️⃣ Elegir el dataset óptimo
best_dataset = 1
X_train_best = TRAIN_DATASETS[best_dataset]
X_test_best = TEST_DATASETS[best_dataset]

# 3️⃣ Hacer predicciones
y_pred_train = final_model.predict(X_train_best)
y_pred_test = final_model.predict(X_test_best)

# 4️⃣ Evaluar el rendimiento
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Train Accuracy: 0.7980
Test Accuracy: 0.6818


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
# Asumiendo que ya tienes y_test y y_pred (y_prob si calculas ROC AUC)
acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6818181818181818
Precision: 0.7450980392156863
Recall: 0.7676767676767676
F1 Score: 0.7562189054726368
Confusion Matrix:
 [[29 26]
 [23 76]]
